In [1]:
import tensorflow

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지

from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [2]:
#서버용 그래픽 카드 지정
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [3]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("./input/emotion-project-2/train/")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("./input/emotion-project-2/train/"+img)))
    for i in path:
        image = cv2.imread("./input/emotion-project-2/train/"+img+'/'+i)
        image = cv2.resize(image, (48,48))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['neutral', 'fear', 'happy', 'sad', 'angry', 'surprise']


In [4]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0 0]


In [5]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)

(50890, 48, 48, 3) (12723, 48, 48, 3)


In [6]:
datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)
datagen.fit(xtrain)

In [7]:
xtrain[0].shape

(48, 48, 3)

In [8]:
model_d=DenseNet201(weights='imagenet',include_top=False, input_shape=(48, 48, 3)) #채널이 무조건 3개여야 하며 크기는 최소 32 이상

x=model_d.output

x= GlobalAveragePooling2D()(x) #전역 평균 풀링 레이어 추가
x= BatchNormalization()(x) #배치 정규화 레이어
x= Dropout(0.5)(x)
#Fully Connected 레이어 추가
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x) #과적합 감소용 드롭아웃 레이어

output=Dense(6,activation='softmax')(x) # because we have to predict the AUC
model=Model(model_d.input,output)

In [9]:
model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 24, 24, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [10]:
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=32),
               epochs=100,
               validation_data=(xtest, ytest))

Epoch 1/100
1591/1591 [==============================] - 114s 72ms/step - loss: 2.0331 - accuracy: 0.3118 - val_loss: 1.4514 - val_accuracy: 0.4797
Epoch 2/100
1591/1591 [==============================] - 109s 69ms/step - loss: 1.5234 - accuracy: 0.4685 - val_loss: 1.1811 - val_accuracy: 0.5585
Epoch 3/100
1591/1591 [==============================] - 110s 69ms/step - loss: 1.3499 - accuracy: 0.5306 - val_loss: 1.2495 - val_accuracy: 0.5485
Epoch 4/100
1591/1591 [==============================] - 109s 69ms/step - loss: 1.2679 - accuracy: 0.5615 - val_loss: 1.3109 - val_accuracy: 0.6050
Epoch 5/100
1591/1591 [==============================] - 110s 69ms/step - loss: 1.2161 - accuracy: 0.5784 - val_loss: 1.6380 - val_accuracy: 0.5912
Epoch 6/100
1591/1591 [==============================] - 112s 70ms/step - loss: 1.0847 - accuracy: 0.6197 - val_loss: 1.2593 - val_accuracy: 0.6484
Epoch 7/100
1591/1591 [==============================] - 111s 70ms/step - loss: 0.9467 - accuracy: 0.6530 - val_

In [11]:
model.evaluate(xtest, ytest, batch_size = 32)

398/398 [==============================] - 9s 23ms/step - loss: 0.5966 - accuracy: 0.8830


[0.5966267585754395, 0.8830464482307434]

In [12]:
#model.save('Densenet201_keras_emotion6ver_imsize48__lr0.001_epoch100.h5')

In [13]:
#완성도 확인
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]): #가장 큰 값을 찾아 인덱스 반환
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

Total-test-data; 12723 	accurately-predicted-data: 11235 	 wrongly-predicted-data:  1488
Accuracy: 88.305 %


In [1]:
plt.title("model accuracy")
#plt.figure(10,20)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.axis([1, 100, 0, 1])
plt.ylabel("accuracy", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel("epoch", fontsize=15)
plt.legend(["train", "validation"], loc="lower right") 
plt.show()

NameError: name 'plt' is not defined

In [24]:
plt.savefig('Denseent201_e100_class4.png')

<Figure size 432x288 with 0 Axes>